In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

X_train = np.load('Xtrain_Classification1.npy') #6254 x 2352. 2352 = pixels x pixels x 3 (RGB). Training set is inbalanced, different number of samples for each class
y_train = np.load('ytrain_Classification1.npy') #6254. 1D vector
X_test = np.load('Xtest_Classification1.npy') #1764 x 2352. Has data from two distinct sourcesprint(X_train[0:10,0:2], y_train[0:200])

We should scaler down the values of the training data by dividing each value with 255 (highest value of "brightness" of pixel) since classifiers can be sensitive to feature scaling. In this case, we will standardize it.

In [2]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) #apply same transformation to test data

We need to define possible values of alpha, the regularization term in the MLP classifier function that prevents overfitting.

In [3]:
def create_lambdas(begin, end, increment):
    """ Creates a list of hyperparameter values for cross-validation. """
    return [begin + i * increment for i in range(int((end - begin) / increment))]

In [4]:
alphas_list = create_lambdas(0.1, 10, 1)
lrate_init = create_lambdas(0.0001, 0.001, 0.0001)
parameters = {'alpha': [0.5, 1, 1.5, 2], 'learning_rate_init': [0.0001, 0.0005, 0.001, 0.01, 0.1]} #To add to GridSearchCV we need to create a dictionary

‘adam’ refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy Ba

In [ ]:
classifier = MLPClassifier(solver= 'adam') #Keep default solver 'adam' since is better for larger datasets and is robust
grid_search = GridSearchCV(classifier, parameters) 

grid_search.fit(X_train, y_train)

best_estimator = grid_search.best_estimator_
print(best_estimator)